In [4]:
import pandas as pd
import os
import time
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from model import LineageMantaObject
from utils import create_raw_table, transfrom_column_name_before_create_table, convert_df_to_oracle_format
from config import NODE_PATH, LINEAGE_SOURCE, LINEAGE_TARGET,EXPORT_MANTA_FILE_PATH, LINEAGE_MANTA_TABLE_NAME_RAW, TABLE_MANTA_LINEAGE_RAW_COLUMN, LINEAGE_MANTA_TABLE_FLOW_TEST, TABLE_MANTA_LINEAGE_FLOW
from repo import RepoLineage

os.getcwd(), c:\Users\hungpv6\Documents\GitHub\DG_tool\main


In [5]:

if __name__ == '__main__':
    start_time = time.time()
    object = LineageMantaObject(lineage_name= 'manta')
    df_raw, table_name_list = create_raw_table(object)
    df_graph = None
    pd.set_option('display.max_rows', None)
    for table_name in table_name_list:
        df = df_raw.query(f"Table_name == '{table_name}' ")
        # df = df_raw.loc[df_raw['TABLE_NAME'] == table_name]
        df_describe = object.describe_table(
            df_path=df,
            source_col=LINEAGE_SOURCE,
            target_col=LINEAGE_TARGET,

        )
        all_values = list(df_describe['all_values'])
        redact_node_df = object.redact_node_with_int(total_nodes_df=df)
        combo_list = redact_node_df[NODE_PATH].tolist()
        dictionary = object.mapping_dict_nodepath(df =redact_node_df)
        adjacency_list = object.create_adjacency_list(dictionary = dictionary )
        combo_list_copy = combo_list.copy()
        
        graph = object.deploy_lineage_manta(df_raw= df, 
                                            combo_list_copy=combo_list_copy, 
                                            dictionary=dictionary,
                                            adjacency_list = adjacency_list, 
                                            source_col=LINEAGE_SOURCE, 
                                            target_col=LINEAGE_TARGET)
        object.add_name_of_table_column(df = graph, table_name = table_name)
        if df_graph is None:
            df_graph = graph
        else:
            df_graph = pd.concat([df_graph, graph])
        end_time = time.time()
        if (int(end_time) - int(start_time)) >= 300:
            print(f"Time elapsed --- need impove the algorithm: {end_time - start_time} seconds")
            break
    # df_graph = object.extract_components(table = df_graph, column_extract='Raw_Node')
    # df_graph = transfrom_column_name_before_create_table(df=df_graph)


LINEAGE_MANTA_TEST
TABLE_NAME =  ACCOUNT | FILE_NAME=  relations_ACCOUNT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_ACCOUNT.csv


In [6]:
print(type(df_graph))
print(df_graph.head(10))
print(len(df_graph))

<class 'pandas.core.frame.DataFrame'>
                                  Flow_implement_raw  \
0  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...   
1  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...   
2  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...   
3  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...   
4  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...   
5  Oracle.W4DBN4.HDB.YACCOUNT.YACCOUNT_BODY.<6,1>...   
6  Oracle.W4DBN4.HDB.ACCOUNT.REMOTE_VERSION,Oracl...   
7  Oracle.W4DBN4.HDB.ACCOUNT.REMOTE_VERSION,Oracl...   
8  Oracle.W4DBN4.HDB.ACCOUNT.REMOTE_VERSION,Oracl...   
9  Oracle.W4DBN4.HDB.ACCOUNT.REMOTE_VERSION,Oracl...   

                                            Raw_Node      ValueObjectType  \
0  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...  PLSQL VarAssignment   
1  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...  PLSQL VarAssignment   
2  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,...  PLSQL VarAssignment   
3  Oracle.W4DBN4.HDB.YTACCOUNT.YTACCO

In [ ]:
# print(df_graph.head(10)) 

In [ ]:
database = 'test'
enviroment = 'DEV'
db_connection = RepoLineage()
db_connect = db_connection.get_connection(db_name='test',environment = 'DEV', user = 'dtc')
# data = db_connection.create_lineage_manta_table(connection = db_connect,
#                                                 table_name = LINEAGE_MANTA_TABLE_NAME_RAW,
#                                                 column_definitions= TABLE_MANTA_LINEAGE_RAW_COLUMN)
# db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_NAME_RAW, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_raw)))

In [ ]:
data = db_connection.create_lineage_manta_table(connection = db_connect,
                                                table_name = LINEAGE_MANTA_TABLE_FLOW_TEST,
                                                column_definitions= TABLE_MANTA_LINEAGE_FLOW)



In [ ]:
db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_FLOW_TEST, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_graph)))

In [ ]:
# data = db_connection.get_data(connection = db_connect, query = f"SELECT * FROM {LINEAGE_MANTA_TABLE_NAME_RAW} WHERE ROWNUM <= 1000")
# print(data)

In [ ]:
data = db_connection.get_data(connection = db_connect, query = f"SELECT * FROM {LINEAGE_MANTA_TABLE_FLOW_TEST} WHERE ROWNUM <= 10")
print(data)

In [ ]:
# db_connection.drop_table(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_FLOW_TEST)

In [ ]:



# table = extract_components(graph,'Raw_Node')
# # print(table)

In [3]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, path):
        """ Thêm nguyên đường đi vào Trie """
        node = self.root
        for elem in path:
            if elem not in node.children:
                node.children[elem] = TrieNode()
            node = node.children[elem]
        node.is_end = True

    def is_superset(self, path):
        """ Kiểm tra xem có đường đi nào lớn hơn path đã tồn tại không """
        node = self.root
        for elem in path:
            if elem not in node.children:
                return False  # Nếu có phần tử không khớp, chắc chắn không phải tập cha
            node = node.children[elem]
        return True  # Nếu đã có đường đi lớn hơn thì bỏ qua

def remove_subsets_trie(paths):
    """ Xóa bỏ các đường đi là tập con bằng Trie """
    paths.sort(key=len, reverse=True)  # Sắp xếp theo độ dài giảm dần
    trie = Trie()
    unique_paths = []
    
    for path in paths:
        path_tuple = tuple(path)  # Chuyển thành tuple để dễ so sánh
        if not trie.is_superset(path_tuple):  # Nếu chưa có đường đi lớn hơn, giữ lại
            trie.insert(path_tuple)
            unique_paths.append(path)
    
    return unique_paths

# 🔥 Test case với dữ liệu có dạng tuple (step, node_id, neighbor_id)
graph = [
    [(0, 1, 0)],
    [(0, 2, 0)],
    [(0, 22, 0)],
    [(0, 2, 0), (1, 2, 1868)],  
    [(0, 22, 0), (1, 22, 282)],  
    [(0, 2, 0), (1, 2, 1868), (2, 1868, 72)],  
]

filtered_paths = remove_subsets_trie(graph)
print(filtered_paths)


[[(0, 2, 0), (1, 2, 1868), (2, 1868, 72)], [(0, 22, 0), (1, 22, 282)], [(0, 1, 0)]]
